In [2]:
import math
import re
from itertools import zip_longest

import numpy as np
from aocd.models import Puzzle
import pickle
import pyperclip

from collections import deque
from functools import cache

In [3]:
year, day = 2024, 11

In [4]:
puzzle = Puzzle(year=year, day=day)

In [5]:
puzzle.examples[0].answer_a

'55312'

In [6]:
example = puzzle.examples[0].input_data
print(example)

Initial arrangement:
125 17

After 1 blink:
253000 1 7

After 2 blinks:
253 0 2024 14168

After 3 blinks:
512072 1 20 24 28676032

After 4 blinks:
512 72 2024 2 0 2 4 2867 6032

After 5 blinks:
1036288 7 2 20 24 4048 1 4048 8096 28 67 60 32

After 6 blinks:
2097446912 14168 4048 2 0 2 4 40 48 2024 40 48 80 96 2 8 6 7 6 0 3 2


In [7]:
def solution_a(data: str, steps: int) -> tuple[str, str, int]:
    # The FIRST applicable rule applies:
    # If the stone is engraved with the number 0, it is replaced by a stone engraved with the number 1.
    # If the stone is engraved with a number that has an even number of digits, it is replaced by two stones. 
    #   The left half of the digits are engraved on the new left stone, 
    #   and the right half of the digits are engraved on the new right stone. (The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0.)
    # If none of the other rules apply, the stone is replaced by a new stone; 
    #   the old stone's number multiplied by 2024 is engraved on the new stone.
    stones = deque(data.split())

    @cache
    def next_number(stone: str) -> list[str]:
        number = int(stone)
        if number == 0:
            return ["1"]
        elif len(stone) % 2 == 0:
            return [f"{int(stone[:len(stone) // 2])}", f"{int(stone[len(stone) // 2:])}"]
        else:
            return [str(number * 2024)]
        
    for i in range(1, steps+1):
        #print(f"After {i} blink")
        next_evolution = []
        while stones:
            stone = stones.popleft()
            next_evolution.extend(next_number(stone))
            
        stones = deque(next_evolution)
        #print(" ".join(next_evolution))
            
    return stones, " ".join(stones)

    

In [8]:
solution_a("0 1 10 99 999", 1)

(deque(['1', '2024', '1', '0', '9', '9', '2021976']), '1 2024 1 0 9 9 2021976')

In [9]:
%%timeit
solution_a("0 1 10 99 999", 1)

3.83 μs ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [10]:
assert solution_a("125 17", 1)[1] == "253000 1 7"

In [11]:
assert solution_a("125 17", 6)[1] == "2097446912 14168 4048 2 0 2 4 40 48 2024 40 48 80 96 2 8 6 7 6 0 3 2"

In [12]:
answer_a = solution_a(puzzle.input_data, 25)

In [13]:
len(answer_a[0])

233050

In [90]:
puzzle.answer_a = len(answer_a[0])

## Part Two


In [74]:
from collections import Counter, defaultdict


def solution_b(data: str, steps: int) -> tuple[str, str, int]:
    # The FIRST applicable rule applies:
    # If the stone is engraved with the number 0, it is replaced by a stone engraved with the number 1.
    # If the stone is engraved with a number that has an even number of digits, it is replaced by two stones. 
    #   The left half of the digits are engraved on the new left stone, 
    #   and the right half of the digits are engraved on the new right stone. (The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0.)
    # If none of the other rules apply, the stone is replaced by a new stone; 
    #   the old stone's number multiplied by 2024 is engraved on the new stone.
    def get_stone_types(stones: list[str]) -> dict[str, int]:
        return Counter(stones)
    
    @cache
    def next_stones(stone: str) -> list[str]:
        number = int(stone)
        if number == 0:
            return ["1"]
        elif len(stone) % 2 == 0:
            return [f"{int(stone[:len(stone) // 2])}", f"{int(stone[len(stone) // 2:])}"]
        else:
            return [str(number * 2024)]

    def update_position(positions: dict[str, list[int]], stone: str, pos: int):
        next_stones = next_number(stone)
        next_stones.reverse()
        del stones[pos]
        for next_stone in next_stones:
            stones.insert(pos, next_stone)

    stones = data.split()
    stone_types = get_stone_types(stones)
        
    for i in range(1, steps+1): 
        next_evolution = defaultdict(lambda: 0)
        for stone, number in stone_types.items():      
            for next_stone in next_stones(stone):
                next_evolution[next_stone] += number

        stone_types = next_evolution
        print("After {i}. step:", stone_types)
            
    return stone_types

    

In [79]:
solution_b("125 17", 6).values()

After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107b51900>, {'253000': 1, '1': 1, '7': 1})
After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107b50940>, {'253': 1, '0': 1, '2024': 1, '14168': 1})
After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107b51900>, {'512072': 1, '1': 1, '20': 1, '24': 1, '28676032': 1})
After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107b50940>, {'512': 1, '72': 1, '2024': 1, '2': 2, '0': 1, '4': 1, '2867': 1, '6032': 1})
After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107b51900>, {'1036288': 1, '7': 1, '2': 1, '20': 1, '24': 1, '4048': 2, '1': 1, '8096': 1, '28': 1, '67': 1, '60': 1, '32': 1})
After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107b50940>, {'2097446912': 1, '14168': 1, '4048': 1, '2': 4, '0': 2, '4': 1, '40': 2, '48': 2, '2024': 1, '80': 1, '96': 1, '8': 1, '6': 2, '7': 1, '3': 1})


dict_values([1, 1, 1, 4, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1])

In [81]:
answer_b = solution_b(puzzle.input_data, 75)

After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107b51000>, {'77': 1, '25': 1, '374440': 1, '4048': 1, '132': 1, '869': 1, '1': 1, '3725044488': 1, '6': 1, '2': 1, '53251440': 1})
After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107575480>, {'7': 2, '2': 1, '5': 1, '374': 1, '440': 1, '40': 1, '48': 1, '267168': 1, '1758856': 1, '2024': 1, '37250': 1, '44488': 1, '12144': 1, '4048': 1, '5325': 1, '1440': 1})
After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107b51000>, {'14168': 2, '4048': 1, '10120': 1, '756976': 1, '890560': 1, '4': 2, '0': 1, '8': 1, '267': 1, '168': 1, '3559924544': 1, '20': 1, '24': 1, '75394000': 1, '90043712': 1, '24579456': 1, '40': 2, '48': 1, '53': 1, '25': 1, '14': 1})
After {i}. step: defaultdict(<function solution_b.<locals>.<lambda> at 0x107575480>, {'28676032': 2, '40': 1, '48': 1, '20482880': 1, '756': 1, '976': 1, '890': 1, '560': 1, '8096': 2, '1': 2, '16192': 1, '540408': 1, '340

In [83]:
puzzle.answer_b = sum(answer_b.values())

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 11! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
